## Pre-processing

In [1]:
import re
import json
import nltk
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import collections as coll
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

In [2]:
# nltk.download('punkt')
st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
      "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
main_characters = ['ДЖОУИ', 'МОНИКА', 'РЕЙЧЕЛ', 'РОСС', 'ФИБИ', 'ЧЕНДЛЕР']
main_characters = sorted(main_characters)
texts_ru = {i : "" for i in main_characters}
texts_eng = {i : "" for i in main_characters}

for name in tqdm(main_characters):
    url_ru = f'https://raw.githubusercontent.com/Alenush/style_transfer_sirius2021summer/master/data/scripts/russian/{name}.txt'
    req = requests.get(url_ru)
    if req.status_code == requests.codes.ok:
        req = BeautifulSoup(req.text, 'html.parser')
        texts_ru[name] = str(req)
    else:
        print('Content was not found.')
        
    url_eng = f'https://raw.githubusercontent.com/Alenush/style_transfer_sirius2021summer/master/data/scripts/english/{name}.txt'
    req = requests.get(url_eng)      
    if req.status_code == requests.codes.ok:
        req = BeautifulSoup(req.text, 'html.parser')
        texts_eng[name] = str(req)
    else:
        print('Content was not found.')

100%|██████████| 6/6 [00:05<00:00,  1.03it/s]


## Promptly for paper

In [5]:
# amount of english words per character
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
amount_en = dict()
amount_ru = dict()
for name, name_en in tqdm(zip(main_characters, ['Joey', 'Monica', 'Rachel', 'Ross', 'Phoebe', 'Chandler'])):
    text = texts_eng[name]
    amount_en[name_en] = [len(sent_tokenize(text)), len(word_tokenize(text))]
    text = texts_ru[name]
    amount_ru[name_en] = [len(sent_tokenize(text)), len(word_tokenize(text))]

6it [00:17,  2.86s/it]


In [6]:
df_stats = pd.DataFrame([(i, j[0] , j[1]) for i, j in amount_en.items()], 
                        columns=['character','eng_sentences','eng_words'])
df_stats['ru_sentences'] = [(j[0]) for i, j in amount_ru.items()]
df_stats['ru_words'] = [(j[1]) for i, j in amount_ru.items()]
df_stats = df_stats[['character','eng_sentences','eng_words','ru_sentences','ru_words']]
print(df_stats.to_latex(index=False))

\begin{tabular}{lrrrr}
\toprule
character &  eng\_sentences &  eng\_words &  ru\_sentences &  ru\_words \\
\midrule
     Joey &          14590 &     114341 &         14562 &    100582 \\
   Monica &          13228 &     105350 &         13182 &     90818 \\
   Rachel &          15664 &     126285 &         15593 &    110510 \\
     Ross &          15580 &     126339 &         15524 &    109708 \\
   Phoebe &          13053 &     104681 &         12965 &     91313 \\
 Chandler &          13060 &     109603 &         13039 &     95744 \\
\bottomrule
\end{tabular}



## Lexical features 
#### Russian
#### https://github.com/ivbeg/readability.io/wiki/API

In [7]:
texts = texts_ru

metrics = []
for name in tqdm(main_characters):
    url_ru = f'https://raw.githubusercontent.com/Alenush/style_transfer_sirius2021summer/master/data/scripts/russian/{name}.txt'
    response = requests.post(f"http://api.plainrussian.ru/api/1.0/ru/measure/?url={url_ru}")
    metrics.append((name, response.json()))
metrics = sorted(metrics, key=lambda x: x[1]['indexes']['index_SMOG'])

100%|██████████| 6/6 [00:08<00:00,  1.47s/it]


In [8]:
metrics_dct = dict(metrics)
complex_words = np.array([(name, int(metrics_dct[name]['metrics']['n_complex_words'])) for name in main_characters])
df_metrics = pd.DataFrame(complex_words)
df_metrics.columns = ['name', 'complex_words']

df_metrics.name = df_metrics.name.str.replace("ДЖОУИ", "Joey")
df_metrics.name = df_metrics.name.str.replace("МОНИКА", "Monica")
df_metrics.name = df_metrics.name.str.replace("РЕЙЧЕЛ", "Rachel")
df_metrics.name = df_metrics.name.str.replace("РОСС", "Ross")
df_metrics.name = df_metrics.name.str.replace("ФИБИ", "Phoebe")
df_metrics.name = df_metrics.name.str.replace("ЧЕНДЛЕР", "Chandler")

df_metrics.complex_words = df_metrics.complex_words.astype(int)
avg_slen  = np.array([(name, int(metrics_dct[name]['metrics']['avg_slen'])) for name in main_characters]).T[1]
df_metrics['avg_slen'] = avg_slen.astype(int)
n_words  = np.array([(name, int(metrics_dct[name]['metrics']['n_words'])) for name in main_characters]).T[1]
df_metrics['n_words'] = n_words.astype(int)
n_sentences  = np.array([(name, int(metrics_dct[name]['metrics']['n_sentences'])) for name in main_characters]).T[1]
df_metrics['n_sentences'] = n_sentences.astype(int) 
c_share  = np.array([(name, metrics_dct[name]['metrics']['c_share']) for name in main_characters]).T[1]
df_metrics['c_share'] = c_share.astype(float)


# fig, axs = plt.subplots(2, 3, figsize=(20,10))
# axs = axs.flatten()

# ax = axs[0]
# ax = sns.barplot(x="name", y="n_words", data=df_metrics, ax=ax)
# ax.set(xlabel='Персонаж', ylabel='Кол-во слов', title='Общее количество слов')

# ax = axs[1]
# ax = sns.barplot(x="name", y="complex_words", data=df_metrics, ax=ax)
# ax.set(xlabel='Персонаж', ylabel='Кол-во слов', title='Количество сложных слов')

# ax = axs[2]
# ax = sns.barplot(x="name", y="avg_slen", data=df_metrics, ax=ax)
# ax.set(xlabel='Персонаж', ylabel='Кол-во слов', title='Среднее число слов в предложении')

# ax = axs[3]
# ax = sns.barplot(x="name", y="n_sentences", data=df_metrics, ax=ax)
# ax.set(xlabel='Персонаж', ylabel='Кол-во предложений', title='Общее количество предложений')
 
# ax = axs[4]
# ax = sns.barplot(x="name", y="c_share", data=df_metrics, ax=ax)
# ax.set(xlabel='Персонаж', ylabel='Доля (%)', title='Доля сложных слов от общего числа')
 
# fig.patch.set_facecolor('white')
    
# plt.show()

In [9]:
def change_width(ax, new_value) :
    for patch in ax.patches :
        current_width = patch.get_width()
        diff = current_width - new_value
        # we change the bar width
        patch.set_width(new_value)
        # we recenter the bar
        patch.set_x(patch.get_x() + diff * .5)

In [10]:
# amount of english words per character
from nltk.tokenize import sent_tokenize
amount_sent = dict()
for name, name_en in tqdm(zip(main_characters, ['Joey', 'Monica', 'Rachel', 'Ross', 'Phoebe', 'Chandler'])):
    text = texts_eng[name]
    amount_sent[name_en] = len(sent_tokenize(text))
np.array(list(amount_sent.values())).mean()

6it [00:01,  3.49it/s]


14195.833333333334

In [11]:
df_metrics['n_sentences_en'] = df_metrics.name.apply(lambda x: amount_sent[x])

## Vocabulary metrics
#### https://github.com/Hassaan-Elahi/Writing-Styles-Classification-Using-Stylometric-Analysis/blob/master/Code/main.py

In [12]:
def hapaxLegemena(words):
    ### only one instance of use
    
    V1 = 0
    # dictionary comprehension . har word kay against value 0 kardi
    freqs = {key: 0 for key in words}
    for word in words:
        freqs[word] += 1
    for word in freqs:
        if freqs[word] == 1:
            V1 += 1
    N = len(words)
    V = float(len(set(words)))
    R = 100 * math.log(N) / max(1, (1 - (V1 / V)))
    h = V1 / N
    return h

def ShannonEntropy(words):
    # -1*sigma(pi*lnpi)
    # Shannon and sympsons index are basically diversity indices for any community
    
    lenght = len(words)
    freqs = coll.Counter()
    freqs.update(words)
    arr = np.array(list(freqs.values()))
    distribution = 1. * arr
    distribution /= max(1, lenght)
    import scipy as sc
    H = sc.stats.entropy(distribution, base=2)
    # H = sum([(i/lenght)*math.log(i/lenght,math.e) for i in freqs.values()])
    return H

def YulesCharacteristicK(words):
    # K  10,000 * (M - N) / N**2
    # , where M  Sigma i**2 * Vi.
    N = len(words)
    freqs = coll.Counter()
    freqs.update(words)
    vi = coll.Counter()
    vi.update(freqs.values())
    M = sum([(value * value) * vi[value] for key, value in freqs.items()])
    K = 10000 * (M - N) / math.pow(N, 2)
    return K

In [13]:
%%time
voc_metrics = []
for name in main_characters:
    text = nltk.tokenize.word_tokenize(texts[name])
    words = [word for word in text if word not in st]
    
    voc_metrics_name = [name]
    voc_metrics_name.append(hapaxLegemena(words))
    voc_metrics_name.append(ShannonEntropy(words))
    voc_metrics_name.append(YulesCharacteristicK(words))
    voc_metrics.append(voc_metrics_name)
df_voc_metrics = pd.DataFrame(voc_metrics)
df_voc_metrics.columns = ['name', 'hapaxLegemena', 'ShannonEntropy', 'Yules']
df_voc_metrics

<ipython-input-12-730e812bb635>:42: RuntimeWarning: overflow encountered in long_scalars
  K = 10000 * (M - N) / math.pow(N, 2)


Wall time: 7.07 s


,name,hapaxLegemena,ShannonEntropy,Yules
0,ДЖОУИ,0.093613,10.273897,0.243041
1,МОНИКА,0.099345,10.253189,0.305831
2,РЕЙЧЕЛ,0.081088,9.990844,0.168498
3,РОСС,0.093483,10.271940,0.256537
4,ФИБИ,0.104359,10.251260,0.295524
5,ЧЕНДЛЕР,0.103805,10.333549,-0.349289


## Readability metrics
#### https://github.com/Hassaan-Elahi/Writing-Styles-Classification-Using-Stylometric-Analysis/blob/master/Code/main.py

##### Upd: расчет Readability index по ссылке не подходит для русского 

In [11]:
# cmuDictionary = None

# def syllable_count_Manual(word):
#     word = word.lower()
#     count = 0
#     vowels = "ауоыиэяюёе"
#     if word[0] in vowels:
#         count += 1
#     for index in range(1, len(word)):
#         if word[index] in vowels and word[index - 1] not in vowels:
#             count += 1
#             if word.endswith("e"):
#                 count -= 1
#     if count == 0:
#         count += 1
#     return count

# def syllable_count(word):
#     global cmuDictionary
#     d = cmuDictionary
#     try:
#         syl = [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]][0]
#     except:
#         syl = syllable_count_Manual(word)
#     return syl

# def FleschReadingEase(words, NoOfsentences):
#     l = float(len(words))
#     scount = 0
#     for word in words:
#         scount += syllable_count(word)

#     I = 206.835 - 1.015 * (l / float(NoOfsentences)) - 84.6 * (scount / float(l))
#     return I

# # def dale_chall_readability_formula(words, NoOfSectences):
# #     difficult = 0
# #     adjusted = 0
# #     NoOfWords = len(words)
# #     with open('dale-chall.pkl', 'rb') as f:
# #         fimiliarWords = pickle.load(f)
# #     for word in words:
# #         if word not in fimiliarWords:
# #             difficult += 1
# #     percent = (difficult / NoOfWords) * 100
# #     if (percent > 5):
# #         adjusted = 3.6365
# #     D = 0.1579 * (percent) + 0.0496 * (NoOfWords / NoOfSectences) + adjusted
# #     return D

# def GunningFoxIndex(words, NoOfSentences):
#     NoOFWords = float(len(words))
#     complexWords = 0
#     for word in words:
#         if (syllable_count(word) > 2):
#             complexWords += 1

#     G = 0.4 * ((NoOFWords / NoOfSentences) + 100 * (complexWords / NoOFWords))
#     return G

# # winSize = 10
# # voc_metrics = []
# # for name in main_characters:
# #     text = nltk.tokenize.word_tokenize(texts[name])
# #     words = [word for word in text if word not in st]
    
# #     voc_metrics_name = [name]
# #     voc_metrics_name.append(FleschReadingEase(words, winSize))
# # #     voc_metrics_name.append(dale_chall_readability_formula(words, winSize))
# #     voc_metrics_name.append(GunningFoxIndex(words, winSize))
# #     voc_metrics.append(voc_metrics_name)
# # df_voc_metrics = pd.DataFrame(voc_metrics)
# # df_voc_metrics.columns = ['name', 'Flesch', 'GunFox']
# # df_voc_metrics

## Sentiment analysis
#### https://github.com/cyberdh/Text-Analysis/tree/master/VADERSentimentAnalysis
##### Смотрим соотношение положительной/ негативной лексики у персонажей.
##### NB: используем скрипты на английском

In [14]:
!git clone https://github.com/Alenush/style_transfer_sirius2021summer.git

Cloning into 'style_transfer_sirius2021summer'...
Updating files:  79% (529/664)
Updating files:  80% (532/664)
Updating files:  81% (538/664)
Updating files:  82% (545/664)
Updating files:  83% (552/664)
Updating files:  84% (558/664)
Updating files:  85% (565/664)
Updating files:  86% (572/664)
Updating files:  87% (578/664)
Updating files:  88% (585/664)
Updating files:  89% (591/664)
Updating files:  90% (598/664)
Updating files:  91% (605/664)
Updating files:  92% (611/664)
Updating files:  93% (618/664)
Updating files:  94% (625/664)
Updating files:  95% (631/664)
Updating files:  96% (638/664)
Updating files:  97% (645/664)
Updating files:  98% (651/664)
Updating files:  99% (658/664)
Updating files: 100% (664/664)
Updating files: 100% (664/664), done.


In [25]:
!cd style_transfer_sirius2021summer
!git checkout master
!git pull

In [26]:
!cd
!cd data/data_for_tone_analysis

In [12]:
posWords = pd.read_csv("posWords.csv", index_col=False).iloc[:, 1:]
pos_list = posWords.values.T[0] 
negWords = pd.read_csv("negWords.csv", index_col=False).iloc[:, 1:]
neg_list = negWords.values.T[0] 

In [13]:
en_text = {name : [] for name in main_characters}
en_text['НЕДРУГ'] = []

for i in tqdm(range(1, 11)):
    url = f'https://github.com/Alenush/style_transfer_sirius2021summer/tree/master/data/scripts/english/{i}'
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    for j in soup.find_all('span', {"class": "css-truncate css-truncate-target d-block width-fit"}):
        url = f'https://raw.githubusercontent.com' + j.find('a')['href'].replace('blob/', '') 
        req = requests.get(url)
        if req.status_code == requests.codes.ok:
            soup = str(BeautifulSoup(req.text, 'html.parser'))
        else:
            print('Content was not found.')
        script = []
        for line in soup.split('\n'):
            if len(line) != 0 and line[0].isupper():
                tag = line.find(":")
                script.append((line[:tag], line[tag + 1:].strip()))
        for person in set([i[0] for i in script]):
            words = [i[1] for i in script if i[0] == person]
            en_text[person] += words

100%|██████████| 10/10 [02:33<00:00, 15.36s/it]


In [14]:
ch = sorted(list(set([i[0] for i in script])))
en_bags = en_text.copy()
for person in ch:
    # person = ch[0]
    seq = "\n".join(en_text[person])
    seq = re.sub("<(.*?)>", "", seq)
    for i in st:
        if i == "'":
            continue
        if i in ('(', ')', '*', '+', '?', '[', '\\', '^'):
            seq = re.sub('[\\' + f'{i}]', " ", seq)
        else:
            seq = re.sub(f'[{i}]', " ", seq)
        seq = re.sub(" +", " ", seq)
    seq = seq.lower()
    seq = seq.split()

    lemmatizer = WordNetLemmatizer()
    seq_lem = [lemmatizer.lemmatize(s, pos='v') for s in seq]
    en_bags[person] = seq_lem

In [27]:
sentiment_res = dict()
for person in ch:
    person_bag = en_bags[person]
    person_pos = {w : person_bag.count(w) for w in pos_list}
    person_neg = {w : person_bag.count(w) for w in neg_list}
    sentiment_res[person] = (person_pos, person_neg)
    
pos = []
neg = []
for name in sentiment_res:
    pos.append(sum(list(sentiment_res[name][0].values())))
    neg.append(sum(list(sentiment_res[name][1].values())))
posneg = np.array([pos, neg])
posneg_sh = posneg[0] / (posneg[0] + posneg[1])

In [ ]:
persons = list(sentiment_res.keys())
index = np.arange(len(persons))
bar_width = 0.35
matplotlib.rcParams.update({'font.size': 10})

fig, axs = plt.subplots(1, 2, figsize=(15,5))
axs = axs.flatten()

ax = axs[0]
pos_plot = ax.bar(index, pos, bar_width, label='полож.', color='b')
neg_plot = ax.bar(index+bar_width, neg, bar_width, label='нег.', color='r')
ax.set_xlabel('Персонаж')
ax.set_ylabel('Кол-во слов')
ax.set_title('Количество полож./ нег. слов')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(persons)
ax.legend()

ax = axs[1]
# ax.bar(persons, (posneg_sh*0+1), label='полож.', color='r')
ax.bar(persons, posneg_sh, label='полож.', color='b')
ax.set_xlabel('Персонаж')
ax.set_ylabel('Доля')
ax.set_title('Доля положительных слов')
ax.set(ylim=(0.7, 0.75))

fig.patch.set_facecolor('white')

plt.show()

In [ ]:
matplotlib.rcParams.update({'font.size': 16})
persons = list(sentiment_res.keys())
index = np.arange(len(persons))
fig, axs = plt.subplots(1, 1, figsize=(15,10), facecolor='w')
bar_width = 0.35
ax = axs

persons_en = ['Joey', 'Monica', 'Other', 'Rachel', 'Ross', 'Phoebe', 'Chandler']

df_tmp = pd.DataFrame([persons_en, pos, neg]).T
df_tmp.columns = ['char', 'pos', 'neg']
df_tmp = df_tmp.sort_values(by=['pos'])
pos_plot = ax.bar(index-bar_width/2, df_tmp.pos, bar_width, label='Positive', color='b')
neg_plot = ax.bar(index+bar_width/2, df_tmp.neg, bar_width, label='Negative', color='r')
ax.set_xlabel('Character')
ax.set_ylabel('Words')
ax.set_title('Amount of positive and negative words')
ax.set_xticks(index)
ax.set_xticklabels(df_tmp.char)
ax.legend()

# for index, row in df_metrics.iterrows():
#     ax.text(row.name, row.n_words, row.n_words, color='black', ha="center", va="bottom")